In [1]:
from keras.preprocessing.text import Tokenizer #Importar liberias necesarias
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re,os
from keras.utils import to_categorical
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
!wget --no-check-certificate \
https://storage.googleapis.com/txt_tesis/txts_extracted.zip \
 -O /tmp/files.zip

--2020-09-22 12:03:34--  https://storage.googleapis.com/txt_tesis/txts_extracted.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 108.177.13.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2887953 (2.8M) [application/zip]
Saving to: ‘/tmp/files.zip’

/tmp/files.zip      100%[===================>]   2.75M  --.-KB/s    in 0.03s   

2020-09-22 12:03:35 (107 MB/s) - ‘/tmp/files.zip’ saved [2887953/2887953]



In [3]:
!unzip /tmp/files.zip -d /tmp/

Archive:  /tmp/files.zip
   creating: /tmp/txt_tesis/
  inflating: /tmp/txt_tesis/acta_comision_2020_01_21_a_comi_justicia.txt  
  inflating: /tmp/txt_tesis/acta_comision_2019_09_04_a_comi_credenciales.txt  
  inflating: /tmp/txt_tesis/acta_pleno_2019_07_15_a_pleno.txt  
  inflating: /tmp/txt_tesis/acta_comision_2019_07_23_a_comi_credenciales.txt  
  inflating: /tmp/txt_tesis/acta_pleno_2019_09_18_a_pleno.txt  
  inflating: /tmp/txt_tesis/acta_comision_2019_10_29_a_comi_justicia.txt  
  inflating: /tmp/txt_tesis/acta_pleno_2019_10_24_a_pleno.txt  
  inflating: /tmp/txt_tesis/acta_comision_2020_01_22_a_comi_canal.txt  
  inflating: /tmp/txt_tesis/acta_comision_2019_09_05_a_comi_credenciales.txt  
  inflating: /tmp/txt_tesis/acta_comision_2019_07_18_a_comi_justicia.txt  
  inflating: /tmp/txt_tesis/acta_comision_2019_07_30_a_comi_justicia.txt  
  inflating: /tmp/txt_tesis/acta_pleno_2019_09_03_a_pleno.txt  
  inflating: /tmp/txt_tesis/acta_pleno_2019_12_04_a_pleno.txt  
  inflating: /tmp

In [4]:
dataPath='/tmp/txt_tesis/'
data=''

In [5]:
txtNames=[]

for txtName in os.listdir(dataPath):
    txtNames.append(txtName)
    
i=0
for txtName in txtNames:
    txtFile=open(dataPath+txtName)
    txtFile.seek(0)
    if(i<=18):
      data=data+txtFile.read()
    i=i+1


In [6]:
len(data)

1254353

In [7]:
cleaned = re.sub(r'\W+', ' ', data).lower()
tokens = word_tokenize(cleaned)
train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

sequences = {}
count = 1

for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
        
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 



In [8]:
vocabulary_size = len(tokenizer.word_counts)+1

In [9]:
n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

In [10]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
#train_inputs.shape
#print(train_targets[0])

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [12]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,epochs=50,verbose=1,callbacks=[checkpoint, reduce, tensorboard_Visualization])
model.save("nextword1.h5")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 3)              39834     
_________________________________________________________________
lstm (LSTM)                  (None, 3, 50)             10800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dense_1 (Dense)              (None, 13278)             677178    
Total params: 750,562
Trainable params: 750,562
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
   1/6289 [..............................] - ETA: 0s - loss: 9.4939 - accuracy: 0.0000e+00WARNING:te

In [13]:
from keras.models import Sequential, load_model
model = load_model("nextword1.h5")

In [31]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

En cuanto a
[7, 349, 6] [[  7 349   6]]
Next word suggestion: la
Next word suggestion: los
Next word suggestion: una
